# Progetto Machine Learning

Obiettivo:  
Creare un modello in grado di prevedere se un passeggero del Titanic sarebbe sopravvissuto o meno al tragico naufragio, in base a caratteristiche come età, sesso, classe di viaggio e altre variabili disponibili nel dataset.

In [ ]:
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# Carico ed importo tutte le librerie necessarie allo svolgimento del progetto, 
# in particolare verranno utilizzate le seguenti librerie:  
# Pandas, sklearn, matplotlib e seaborn

# Utilizzando Pandas, vado a caricare il dataset 'titanic_sub.csv' nella variabile df 

In [ ]:
df= pd.read_csv('titanic_sub.csv')

In [ ]:
df.head()

In [ ]:
df.columns


Preprocessing dei dati

In [ ]:
# Separo il target (la colonna 'Survived') dalle altre features.

In [ ]:
X=df[['Sex', 'Age', 'Pclass', 'Embarked']]
y=df['Survived']

In [ ]:
X.head()

In [ ]:
X.dtypes

In [ ]:
# Vedo che ci sono delle variabili categoriche di tipo 'Object' che devo trasformare per essere "digerite" dal modello.
#  
# Pertanto utilizzo LabelEncoder che, seguendo l'ordine alfabetico, trasformerà la categoria 'Sex' da 'Female'\'Male' a '0'\'1'.
# In generale LabelEncoder viene preferito per trasformazioni binarie o per trasformazioni categoriche ordinali, ovvero dove le categorie da trasformare hanno un loro 'ordine di importanza'

# Per quanto riguarda la categoria 'Embarked' contenente i valori C,S,Q (che si riferisce al porto di imbarco e che non hanno un implicito ordine di importanza) utilizzo get_dummies di Pandas 
# perchè utilizza una codifica one-hot, ovvero trasforma una variabile categoriale non ordinata in più colonne binarie (tante quante sono i valori attribuibili alla feature in esame)

In [ ]:
le=LabelEncoder()
X['Sex']=le.fit_transform(X['Sex'])
X.head()

In [ ]:
X=pd.get_dummies(X, columns=['Embarked'], dtype=int)
X.head()

In [ ]:
# Separo 'X' e 'y' in train set (75% dei dati totali) e test set (25% dei dati totali) 
# impostando 'random_state'=0 per garantire la riproducibilità degli esperimenti.
# Questo ci assicura che lo stesso codice produca sempre lo stesso risultato ogni volta che viene eseguito.

# Suddivido ulteriormente il train set, il 25% dei dati verrà destinato al set di validazione
#  che ci permetterà di:
    #  validare il modello su dati non visti prima di testarlo sul test set, 
    #  andando a sperimentare diversi valori degli iperparametri, così da individuare quello più efficace al fine
    #  di prevenire problemi di fitting regolando la complessità del modello.

In [ ]:
X_train1, X_test, y_train1, y_test= train_test_split(X, y, test_size=0.25, random_state=0)
print(X_train1.shape, y_train1.shape, X_test.shape, y_test.shape)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train1, test_size=0.25, random_state=0)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

Modello: albero di decisione. 

Validare la profondità con i valori: 2, 5, 10, 25, profondità max.  

Metrica di valutazione: Accuratezza.

In [ ]:
# Imposto il modello come 'DecisionTreeClassifier':
# Lo addestro con i dati di 'training' e lo vado a validare su quelli di 'validazione' facendo variare 
# la profondità dell'albero con il parametro k.
# Per ogni valore di k valuto l'accuratezza del modello

In [ ]:
for k in [2,5,10,25,None]:
    tree=DecisionTreeClassifier(max_depth=k, random_state=0)
    tree.fit(X_train,y_train)
    predictions=tree.predict(X_val)
    validation_accuracy=accuracy_score(y_val, predictions)
    print(f'Accuratezza modello per k={k}: {validation_accuracy:.2f}')

In [ ]:
# Ne risulta che l'accuratezza maggiore ce lo dà il valore k=10 con una accuratezza pari a 82%

# imposto quindi il modello finale con il valore trovato dell'iperparametro k e vado a testarlo sui dati nuovi,
#  mai visti dal modello, ovvero sul test set

In [ ]:
best_model=DecisionTreeClassifier(max_depth=10, random_state=0)
best_model.fit(X_train,y_train)
predictions_bm=best_model.predict(X_test)
validation_accuracy_bm=accuracy_score(y_test, predictions_bm)
print(f'Accuratezza modello per k={k}: {validation_accuracy_bm:.2f}')

In [ ]:
# Calcolando l'accuratezza del modello finale mi trovo il valore 79%
#  che è più basso di quanto trovato nei test di validazione ma comunque in linea
#   il che ci fa concludere che il modello si comporta bene sui dati mai visti prima.

In [ ]:
# VISUALIZZAZIONE DELL'ALBERO E INTERPRETAZIONE
from sklearn.tree import plot_tree

plt.figure(figsize=(60,40))
plot_tree(best_model, feature_names=X.columns, class_names=["Non sopravvissuto", "Sopravvissuto"], filled=True)
plt.show()